In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2
import h5py

from pathlib import Path
from sglm.models import sglm_cv
import itertools
import time
import glob
import numpy as np
import pandas as pd
import numpy as np
from sglm.features import gen_signal_df as gsd
from sglm.features import build_features as bf
from sglm.features import setup_model_fit as smf
from sglm.models import sglm_cv
from sglm import models
from sglm.visualization import visualize
from sglm.models import train_model
from sglm.models import eval
from sglm import features
import sys
import os
from pathlib import Path
import matplotlib.pyplot as plt
import json

import traceback
import warnings
import sys

In [2]:
# From stack overflow
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

In [3]:
def extract_multifiles(wt_used, signal_files, widest_orders):
    # needed info
    # Params: wt_used, signal_files, widest_orders, multifile_fit
    # Returns: mouse_names, combo_dfs, combo_fns, X_cols_sftd

    file_ids = [Path(_).parts[-1] for _ in signal_files]
    print('file_ids', file_ids)
    combo_dfs, X_cols_sftd, _ = smf.multi_file_analysis_prep(signal_files, widest_orders, file_ids)
    combo_fns = ['_'.join(wt_used).replace('WT', '').replace('S', '')]
    mouse_names = combo_fns

    return mouse_names, combo_dfs, combo_fns, X_cols_sftd

In [4]:
def dump_dict_to_json(source_dict, json_path):
    with open(str((json_path).resolve()), 'w') as json_file:
        json.dump(source_dict, json_file)

def read_json(json_path):
    with open(str((json_path).resolve()), 'r') as json_file:
        read_json = json.load(json_file)
    return read_json

In [5]:
def id_rows_with_all_cols(dfrel_basis,
                          X_y_pairings,
                          X_cols_sftd, 
                          drop_cols_basis = ['nTrial', 'nTrial_filenum', 'cpn', 'cpx', 'spnnr', 'spxnr', 'spnr', 'spxr',
                                             
                                             'photometryCenterInIndex', 'photometryCenterOutIndex',
                                             'photometrySideInIndexr', 'photometrySideInIndexnr',
                                             'photometrySideOutIndex', 'spnnrOff',
                                             
                                             'photometrySideInIndexAA', 'photometrySideInIndexAa',
                                             'photometrySideInIndexaA', 'photometrySideInIndexaa',
                                             'photometrySideInIndexAB', 'photometrySideInIndexAb',
                                             'photometrySideInIndexaB', 'photometrySideInIndexab',
                                             'sl', 'slOff'
                                            ]):
    full_drop_basis = []
    y_col_lst = []
    for X_y_dct in X_y_pairings:
        full_drop_basis += bf.col_shift_bounds_dict_to_col_list(X_y_dct['X_cols'], X_cols_sftd)
        y_col_lst += [X_y_dct['y_col']]
    y_col_drop_basis = sorted(list(set(y_col_lst)))
    full_drop_basis = sorted(list(set(drop_cols_basis + full_drop_basis + y_col_drop_basis)))

    num_cols_na = (dfrel_basis[full_drop_basis].isna().sum(axis=1))
    num_y_0 = (dfrel_basis[y_col_drop_basis] == 0).sum(axis=1)
    has_all_cols = (num_cols_na == 0)&(num_y_0 == 0)

    return has_all_cols

def warn_with_traceback(message, category, filename, lineno, file=None, line=None):

    log = file if hasattr(file,'write') else sys.stderr
    traceback.print_stack(file=log)
    log.write(warnings.formatwarning(message, category, filename, lineno, line))

warnings.showwarning = warn_with_traceback

In [6]:
def get_holdout_and_cv_idx(df_available, pholdout, folds, pgss, filter_series, id_cols=['nTrial_filenum']):
    holdout_series = models.split_data.holdout_split_by_trial_id(df_available,
                                                                id_cols=id_cols,
                                                                perc_holdout=pholdout)
    cv_idx_lst = models.split_data.cv_idx_by_trial_id(dfrel_basis_has_all_cols[(~holdout_series)&(filter_series)],
                                                             trial_id_columns=id_cols,
                                                             num_folds=folds,
                                                             test_size=pgss)
    return holdout_series, cv_idx_lst

In [7]:
### Part 1
param_set_json_name = 'full_model_param_set.json'

source_data_path = Path(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\interim')
output_path = Path(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\outputs_clean')
preproc_params_name = r'preproc_params.json'
basis_name = r'_basis.csv'
figname = 'Figure_6'
groupid = 'g1'
param_set_id = '-20_+20'
neg_order = -20
pos_order = 20
fix_training = True
num_runs = 10
folds = 10
pholdout = 0.5
pgss = 0.2
score_method = 'r2'
val_test_inx_sel_method = 'irun'

X_y_pairings_lst = []
X_y_pairings_lst += [[
    {'X_cols': {
                'photometryCenterInIndex':(0,0),
#                 'photometryCenterOutIndex':(0,0),
                'photometrySideInIndex':(0,0),
                'photometrySideInIndexr':(0,0),                
                'photometrySideOutIndex':(0,0),
                'sl': (0,0),
                'spnnrOff': (0,0),
               },
     'y_col': 'gGLUr',
     'name': 'base_simple'
     },
    {'X_cols': {
                'photometryCenterInIndex':(0,0),
#                 'photometryCenterOutIndex':(0,0),
                'photometrySideInIndexAA':(0,0),
                'photometrySideInIndexAa':(0,0),
                'photometrySideInIndexaA':(0,0),
                'photometrySideInIndexaa':(0,0),
                'photometrySideInIndexAB':(0,0),
                'photometrySideInIndexAb':(0,0),
                'photometrySideInIndexaB':(0,0),
                'photometrySideInIndexab':(0,0),
                'photometrySideOutIndex':(0,0),
                'sl': (0,0),
                'spnnrOff': (0,0),
               },
     'y_col': 'gGLUr',
     'name': 'base_words'},

]]

# Select hyper parameters for GLM to use for model selection
# Step 1: Create a dictionary of lists for these relevant keywords...
kwargs_iterations = {
    # 'alpha': [0],
    # 'l1_ratio': [0],

#     'alpha': [0.0, 0.01, 0.1, 1.0],
#     'l1_ratio': [0.0, 0.01, 0.1, 1.0],
#     'alpha': [0.0, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0, 1000000.0],
    'alpha': [0.0],
    'l1_ratio': [0.0],
}

# Step 2: Create a dictionary for the fixed keyword arguments that do not require iteration...
kwargs_fixed = {
    'max_iter': 10000,
    'fit_intercept': False
}

dfrr_cols = ['signal_file', 'file_num', 'nTrial', 'nTrial_filenum', 'nEndTrial', 'wi_trial_keep',
#              'nTrial_hard', 'nEndTrial_hard', 'diffTrialNums_hard', 'wi_trial_keep_hard',
             'has_all_cols', 'gDA', 'gACH', 'rDA', 'gGLUr', 'gGLUl',
             'diffTrialNums', 'dupe',
             'photometryCenterInIndex', 'photometryCenterOutIndex',
             'photometrySideInIndexr', 'photometrySideInIndexnr',
             'photometrySideOutIndex', 'spnnrOff', 'sl',

             'photometrySideInIndexAA', 'photometrySideInIndexAa',
             'photometrySideInIndexaA','photometrySideInIndexaa',
             'photometrySideInIndexAB', 'photometrySideInIndexAb',
             'photometrySideInIndexaB','photometrySideInIndexab',
]

In [8]:
# Step 3: Generate iterable list of keyword sets for possible combinations
glm_kwarg_lst = sglm_cv.generate_mult_params(kwargs_iterations, kwargs_fixed)

In [9]:
### Part 1
param_set_output_path = output_path / Path(figname) / Path(groupid) / Path(param_set_id)
param_set_output_path.mkdir(parents=True, exist_ok=False)

preproc_params = read_json(source_data_path / preproc_params_name)
dump_dict_to_json(preproc_params, param_set_output_path / preproc_params_name)

param_set_dct = {
    'source_data_path': str((source_data_path).resolve()),
    'output_path': str((output_path).resolve()),
    'basis_name': basis_name,
    'figname': figname,
    'groupid': groupid,
    'param_set_id': param_set_id,
    'neg_order': neg_order,
    'pos_order': pos_order,
    'fix_training': fix_training,
    'num_runs': num_runs,
    'folds': folds,
    'pholdout': pholdout,
    'pgss': pgss,
    'score_method': score_method,
    'val_test_inx_sel_method': val_test_inx_sel_method,
    'X_y_pairings_lst': X_y_pairings_lst,
    'kwargs_iterations': kwargs_iterations,
    'kwargs_fixed': kwargs_fixed,
    'dfrr_cols': dfrr_cols,
}
dump_dict_to_json(param_set_dct, param_set_output_path / param_set_json_name)

basis = pd.read_csv(source_data_path / basis_name, index_col=0)
relevant_basis = basis[(basis['figname'] == figname)&(basis['groupid'] == groupid)]
data_path = (source_data_path / figname / groupid)
wt_used = basis['mouseid'].tolist()

# Load Signal Data
signal_files = []
mouse_names = []
for inx, basis_row in basis.iterrows():
    signal_file_out = Path(basis_row['signal_file_out'])
    signal_path = data_path / signal_file_out
    signal_files.append(str(signal_path.resolve()))
    mouse_names.append(basis_row['mouseid'])

In [10]:
# plot_width = 2
# plot_rows_lst = [_//plot_width + (_%plot_width > 0)*1 for _ in max_cols_len_lst]

In [11]:
# Part 2: Generate base files to use by all runs and create folders for each run

max_cols_len_lst = [max([len(_['X_cols']) for _ in inner_list]) for inner_list in X_y_pairings_lst]
multi_start = time.time()
for iXyp, X_y_pairings in enumerate(X_y_pairings_lst):

    widest_orders = smf.xy_pairs_to_widest_orders([{'X_cols': smf.X_cols_dict_to_default(_['X_cols'], neg_order, pos_order),
                                                    'y_col': _['y_col']} for _ in X_y_pairings])
    max_cols_len = max_cols_len_lst[iXyp]
    mouse_names, combo_dfs, combo_fns, X_cols_sftd = extract_multifiles(wt_used, signal_files, widest_orders)
    
    start = time.time()
    results_dict = {}
    
    dfrel_basis = combo_dfs[0].reset_index(drop=False).copy()
    
#     fn = Path(combo_fns[file_num].split('.')[0]).parts[-1]
    
    mouse_id = mouse_names[0]
    dfresids_cols = np.copy(dfrr_cols).tolist()
#     run_id = f'{fn}_{iXyp}'         
    has_all_cols = id_rows_with_all_cols(dfrel_basis, X_y_pairings, X_cols_sftd)
    if has_all_cols.sum() == 0:
        print(f'No datapoints found for non-NaN dropcols & non-zero ycols for fixed_training: {prefix}_{fn}')
        continue

    dfrel_basis['has_all_cols'] = has_all_cols
    
    Xyp_path = param_set_output_path / Path(f'iXyp_{iXyp}')
    Xyp_path.mkdir(parents=True, exist_ok=False)
    dump_dict_to_json(X_y_pairings, Xyp_path / Path('X_y_pairings.json'))
    
#     dfrel_basis[dfresids_cols].set_index(['nTrial_filenum'], append=True).to_hdf(str((Xyp_path / Path(f'combo_df.h5')).resolve()), key='combo', index=True,)
    dfrel_basis[dfresids_cols].set_index(['nTrial_filenum'], append=True).to_csv(str((Xyp_path / Path(f'combo_df.csv')).resolve()), index=True)
    
    dfrel_basis_has_all_cols = dfrel_basis[dfrel_basis['has_all_cols']]
    
    cv_holdout_idx_params = dict(df_available=dfrel_basis_has_all_cols,
                                 pholdout=pholdout,
                                 folds=folds,
                                 pgss=pgss,
                                 filter_series=dfrel_basis_has_all_cols['wi_trial_keep'],
                                 id_cols=['nTrial_filenum'])
    
    for irun in range(num_runs):
        holdout_irun, cv_idx_irun = get_holdout_and_cv_idx(**cv_holdout_idx_params)
        
        for iXyd, X_y_dct in enumerate(X_y_pairings):
            holdout_iXyd, cv_idx_iXyd = get_holdout_and_cv_idx(**cv_holdout_idx_params)
            
            for igkwargs, glm_kwargs in enumerate(glm_kwarg_lst):
                holdout_igkwargs, cv_idx_igkwargs = get_holdout_and_cv_idx(**cv_holdout_idx_params)
                
                run_path = Xyp_path / f'iXyd_{iXyd}-ikwargs_{igkwargs}' / f'irun_{irun}'
                run_path.mkdir(parents=True, exist_ok=False)
                
                specific_model_params = {
                    'irun': irun,
                    'iXyd': iXyd,
                    'igkwargs': igkwargs,
                    'X_y_dct': X_y_dct,
                    'glm_kwargs': glm_kwargs,
                }
                dump_dict_to_json(specific_model_params, run_path / Path('specific_model_params.json'))
                dump_dict_to_json(param_set_dct, run_path / param_set_json_name)
                
                holdout_df = pd.DataFrame({'holdout_irun': holdout_irun,
                                           'holdout_iXyd':holdout_iXyd,
                                           'holdout_igkwargs':holdout_igkwargs},
                                          index=dfrel_basis.index)
                
#                 holdout_df.to_hdf(str((run_path / 'holdout_options.hd5').resolve()), key='holdout')
                
                cv_options_dct = {
                    'cv_idx_irun':cv_idx_irun,
                    'cv_idx_iXyd':cv_idx_iXyd,
                    'cv_idx_igkwargs':cv_idx_igkwargs,
                }
#                 np.save(str((run_path/'cv_options.npy').resolve()), np.array(cv_options_dct, dtype='object'))
                
                kfold_cv_idx = cv_options_dct[f'cv_idx_{val_test_inx_sel_method}']
                holdout = holdout_df[f'holdout_{val_test_inx_sel_method}']
                
                np.save(str((run_path/'kfold_cv_idx.npy').resolve()), np.array(kfold_cv_idx, dtype='object'))
                holdout.to_hdf(str((run_path / 'holdout.hd5').resolve()), key='holdout')
                
                
                
                #### JZ TODO: MOVE TO SEPARATE FILE FOR PARALLELIZATION OVER DEEPEST FOLDERS...
                #### JZ TODO: MOVE TO SEPARATE FILE FOR PARALLELIZATION OVER DEEPEST FOLDERS...
                #### JZ TODO: MOVE TO SEPARATE FILE FOR PARALLELIZATION OVER DEEPEST FOLDERS...
                #### JZ TODO: MOVE TO SEPARATE FILE FOR PARALLELIZATION OVER DEEPEST FOLDERS...
                #### JZ TODO: MOVE TO SEPARATE FILE FOR PARALLELIZATION OVER DEEPEST FOLDERS...
                #### JZ TODO: MOVE TO SEPARATE FILE FOR PARALLELIZATION OVER DEEPEST FOLDERS...
                #### JZ TODO: MOVE TO SEPARATE FILE FOR PARALLELIZATION OVER DEEPEST FOLDERS...
                #### JZ TODO: MOVE TO SEPARATE FILE FOR PARALLELIZATION OVER DEEPEST FOLDERS...
                #### JZ TODO: MOVE TO SEPARATE FILE FOR PARALLELIZATION OVER DEEPEST FOLDERS...
                #### JZ TODO: MOVE TO SEPARATE FILE FOR PARALLELIZATION OVER DEEPEST FOLDERS...
                #### JZ TODO: MOVE TO SEPARATE FILE FOR PARALLELIZATION OVER DEEPEST FOLDERS...
                #### JZ TODO: MOVE TO SEPARATE FILE FOR PARALLELIZATION OVER DEEPEST FOLDERS...

                X_cols = bf.col_shift_bounds_dict_to_col_list(X_y_dct['X_cols'], X_cols_sftd)
                y_col = X_y_dct['y_col']
                name = X_y_dct['name']


                df_train = dfrel_basis_has_all_cols[(~holdout)&(dfrel_basis_has_all_cols['wi_trial_keep'])]
                df_holdout = dfrel_basis_has_all_cols[(holdout)&(dfrel_basis_has_all_cols['wi_trial_keep'])]

                X_train, y_train, X_holdout, y_holdout = df_train[X_cols], df_train[y_col], df_holdout[X_cols], df_holdout[y_col]
                best_score, best_score_std, best_params, best_model, cv_results = models.sglm_cv.simple_cv_fit(X_train, y_train, kfold_cv_idx, [glm_kwargs], model_type='Normal',
                                                                                                            verbose=0, score_method=score_method)


                print('Variable Sizes:')
                for vname, size in sorted(((vname, sys.getsizeof(value)) for vname, value in locals().items()),
                                        key= lambda x: -x[1])[:30]:
                    print("{:>30}: {:>8}".format(vname, sizeof_fmt(size)))

                glm, holdout_score, holdout_neg_mse_score = eval.training_fit_holdout_score(X_train, y_train, X_holdout, y_holdout, best_params)

                betas_df = pd.DataFrame(np.concatenate([np.array([glm.intercept_]), glm.coef_], axis=0).reshape(1,-1),
                            index=[run_id], columns=['int']+X_cols)
                betas_df['mouse_id'] = mouse_id
                betas_df['channel_name'] = f'{y_col}_{iXyd}_run_num={irun}'
                betas_df['name'] = name

                assert len(cv_results['full_cv_results']) == 1
                assert holdout_neg_mse_score == glm.neg_mse_score(X_holdout, y_holdout)

                betas_df[['mse_tr', 'mse_cv', 'mse_te']] = [[-glm.neg_mse_score(X_train, y_train), cv_results['full_cv_results'][0]['cv_mse_score'], -glm.neg_mse_score(X_holdout, y_holdout)]]
                betas_df[['r2_tr', 'r2_cv', 'r2_te']] = [[glm.r2_score(X_train, y_train), cv_results['full_cv_results'][0]['cv_R2_score'], glm.r2_score(X_holdout, y_holdout)]]

                multi_end = time.time()
                time_passed = str(multi_end - multi_start) + ' s'
                betas_df['timestamp'] = time_passed
                print('run_id, subrun_id', run_id, subrun_id, f'- Time Passed: {time_passed}')

                betas_df = betas_df.set_index(['mouse_id', 'channel_name', 'name', 'timestamp', 'mse_tr', 'mse_cv', 'mse_te', 'r2_tr', 'r2_cv', 'r2_te'], append=True)

                word_perc_kwargs = dict(total_col='photometrySideInIndex', words_prefix='photometrySideInIndex', words=['AA', 'Aa', 'aA', 'aa', 'AB', 'Ab', 'aB', 'ab'])
                betas_df = get_perc_words(df_train, betas_df, perc_suffix='_cnt_tr',  **word_perc_kwargs)
                betas_df = get_perc_words(df_holdout, betas_df, perc_suffix='_cnt_ho', **word_perc_kwargs)

                betas_df.to_hdf(pr(f'{base_folder}/{prefix}/{best_coefs_folder}/{run_id}_{hyp_str}_best_coeffs.h5'), key=subrun_id, index=True)

                dfrel_basis['pred_'+pred_col_name] = pd.Series(glm.predict(dfrel_basis_has_all_cols[X_cols]),
                                                                            index=dfrel_basis_has_all_cols.index)
                dfrel_basis['predALL_'+pred_col_name] = glm.predict(dfrel_basis[X_cols])

                subset_dfresids_cols = [f'holdout_{pred_col_name}', 'pred_'+pred_col_name, 'predALL_'+pred_col_name]
                dfresids_cols += subset_dfresids_cols
                dfrel_basis[dfresids_cols].set_index(['nTrial_filenum'], append=True)[subset_dfresids_cols].to_hdf(pr(f'{base_folder}/{prefix}/{best_reconstruct_folder}/best_resids_{run_id}.h5'), key=subrun_id, index=True,)

                
                
                break
            break
        break
    break
    
                
    
#     dfrel_basis[dfresids_cols].set_index(['nTrial_filenum'], append=True).to_hdf(pr(f'{base_folder}/{prefix}/{best_reconstruct_folder}/best_resids_{run_id}.h5'), key='dfrel_basis', index=True,)
#     dfrel_basis_has_all_cols = dfrel_basis[has_all_cols]
    
    
#     # Define on a Per Run Basis
#     holdout = models.split_data.holdout_split_by_trial_id(dfrel_basis_has_all_cols,
#                                                         id_cols=['nTrial_filenum'],
#                                                         perc_holdout=pholdout)
#     dfrel_basis[f'holdout_iXyp={iXyp}_irun={irun}'] = holdout
#     dfrel_basis[f'holdout_iXyp={iXyp}_irun={irun}'] = dfrel_basis[f'holdout_iXyp={iXyp}_irun={irun}'].astype(float)
    
    
#     for iXyd, X_y_dct in enumerate(X_y_pairings):

#         for glm_kwargs in glm_kwarg_lst:
#             print(glm_kwargs)
#             hyp_str = '__'.join([''.join([str(__) for __ in _]) for _ in glm_kwargs.items()]).replace('.', '____')


#             X_cols = bf.col_shift_bounds_dict_to_col_list(X_y_dct['X_cols'], X_cols_sftd)
#             y_col = X_y_dct['y_col']
#             name = X_y_dct['name']

#             subrun_id = f'{y_col}_{iXyp}_{iXyd}_{name}_{hyp_str}_run_num__{irun}'

#             pred_col_name = subrun_id


#             if not fix_training:
#                 holdout = models.split_data.holdout_split_by_trial_id(dfrel_basis_has_all_cols, id_cols=['nTrial_filenum'], perc_holdout=pholdout)
#                 dfrel_basis[f'holdout_{pred_col_name}'] = holdout
#                 dfrel_basis[f'holdout_{pred_col_name}'] = dfrel_basis[f'holdout_{pred_col_name}'].astype(float)
#             else:
#                 dfrel_basis[f'holdout_{pred_col_name}'] = dfrel_basis[f'holdout_iXyp={iXyp}_irun={irun}'].astype(float)

#             if (~dfrel_basis[f'holdout_{pred_col_name}'].isna()).sum() == 0:
#                 print(f'No datapoints found for non-NaN dropcols & non-zero ycols for run id: {run_id}, {subrun_id}.')
#                 continue

#             df_train = dfrel_basis_has_all_cols[(~holdout)&(dfrel_basis_has_all_cols['wi_trial_keep'])]
#             df_holdout = dfrel_basis_has_all_cols[(holdout)&(dfrel_basis_has_all_cols['wi_trial_keep'])]

#             X_train, y_train, X_holdout, y_holdout = df_train[X_cols], df_train[y_col], df_holdout[X_cols], df_holdout[y_col]

#             kfold_cv_idx = models.split_data.cv_idx_by_trial_id(df_train, trial_id_columns=['nTrial_filenum'], num_folds=folds, test_size=pgss)

#             best_score, best_score_std, best_params, best_model, cv_results = models.sglm_cv.simple_cv_fit(X_train, y_train, kfold_cv_idx, [glm_kwargs], model_type='Normal',
#                                                                                                         verbose=0, score_method=score_method)


#             print('Variable Sizes:')
#             for vname, size in sorted(((vname, sys.getsizeof(value)) for vname, value in locals().items()),
#                                     key= lambda x: -x[1])[:30]:
#                 print("{:>30}: {:>8}".format(vname, sizeof_fmt(size)))

#             glm, holdout_score, holdout_neg_mse_score = eval.training_fit_holdout_score(X_train, y_train, X_holdout, y_holdout, best_params)

#             betas_df = pd.DataFrame(np.concatenate([np.array([glm.intercept_]), glm.coef_], axis=0).reshape(1,-1),
#                         index=[run_id], columns=['int']+X_cols)
#             betas_df['mouse_id'] = mouse_id
#             betas_df['channel_name'] = f'{y_col}_{iXyd}_run_num={irun}'
#             betas_df['name'] = name

#             assert len(cv_results['full_cv_results']) == 1
#             assert holdout_neg_mse_score == glm.neg_mse_score(X_holdout, y_holdout)

#             betas_df[['mse_tr', 'mse_cv', 'mse_te']] = [[-glm.neg_mse_score(X_train, y_train), cv_results['full_cv_results'][0]['cv_mse_score'], -glm.neg_mse_score(X_holdout, y_holdout)]]
#             betas_df[['r2_tr', 'r2_cv', 'r2_te']] = [[glm.r2_score(X_train, y_train), cv_results['full_cv_results'][0]['cv_R2_score'], glm.r2_score(X_holdout, y_holdout)]]

#             multi_end = time.time()
#             time_passed = str(multi_end - multi_start) + ' s'
#             betas_df['timestamp'] = time_passed
#             print('run_id, subrun_id', run_id, subrun_id, f'- Time Passed: {time_passed}')

#             betas_df = betas_df.set_index(['mouse_id', 'channel_name', 'name', 'timestamp', 'mse_tr', 'mse_cv', 'mse_te', 'r2_tr', 'r2_cv', 'r2_te'], append=True)

#             word_perc_kwargs = dict(total_col='photometrySideInIndex', words_prefix='photometrySideInIndex', words=['AA', 'Aa', 'aA', 'aa', 'AB', 'Ab', 'aB', 'ab'])
#             betas_df = get_perc_words(df_train, betas_df, perc_suffix='_cnt_tr',  **word_perc_kwargs)
#             betas_df = get_perc_words(df_holdout, betas_df, perc_suffix='_cnt_ho', **word_perc_kwargs)

#             betas_df.to_hdf(pr(f'{base_folder}/{prefix}/{best_coefs_folder}/{run_id}_{hyp_str}_best_coeffs.h5'), key=subrun_id, index=True)

#             dfrel_basis['pred_'+pred_col_name] = pd.Series(glm.predict(dfrel_basis_has_all_cols[X_cols]),
#                                                                         index=dfrel_basis_has_all_cols.index)
#             dfrel_basis['predALL_'+pred_col_name] = glm.predict(dfrel_basis[X_cols])

#             subset_dfresids_cols = [f'holdout_{pred_col_name}', 'pred_'+pred_col_name, 'predALL_'+pred_col_name]
#             dfresids_cols += subset_dfresids_cols
#             dfrel_basis[dfresids_cols].set_index(['nTrial_filenum'], append=True)[subset_dfresids_cols].to_hdf(pr(f'{base_folder}/{prefix}/{best_reconstruct_folder}/best_resids_{run_id}.h5'), key=subrun_id, index=True,)


file_ids ['GLM_SIGNALS_INTERIM_S1299_02022022.csv', 'GLM_SIGNALS_INTERIM_S1299_02072022.csv', 'GLM_SIGNALS_INTERIM_S1300_02022022.csv', 'GLM_SIGNALS_INTERIM_S1300_02072022.csv', 'GLM_SIGNALS_INTERIM_S1301_01312022.csv', 'GLM_SIGNALS_INTERIM_S1301_02022022.csv', 'GLM_SIGNALS_INTERIM_S1302_01312022.csv', 'GLM_SIGNALS_INTERIM_S1302_02022022.csv']
{'max_iter': 10000, 'fit_intercept': False, 'alpha': 0.0, 'l1_ratio': 0.0}
Running multi
Running multi
Running multi
Running multi
Multirun queue completed
Multirun queue completedMultirun queue completed

{'max_iter': 10000, 'fit_intercept': False, 'alpha': 0.0, 'l1_ratio': 0.0}
> cv_mean_score_train: 0.3731476586264627
> cv_R2_score: 0.37085583843941394
> cv_mean_score: 0.37102745179542324
Running multi
Multirun queue completed
Variable Sizes:
      dfrel_basis_has_all_cols:  1.6 GiB
                   dfrel_basis:  1.6 GiB
                      df_train: 625.5 MiB
                    df_holdout: 610.4 MiB
                       X_train: 224.2 

NameError: name 'run_id' is not defined